In [27]:
'''This script uses OpenCV's haarcascade (face and eye cascade) to detect face
and eyes in a given input image.'''

#Import necessary libraries
import cv2 as cv
import numpy as np

#Load face cascade and hair cascade from haarcascades folder
face_cascade = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_eye.xml")

#Read image in img and convert it to grayscale and store in gray.
#Image is converted to grayscale, as face cascade doesn't require to operate on coloured images.
img = cv.imread('Utils/Maurice.jpg')
img = cv.resize(img, (1000, 1000))
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

if face_cascade.empty():
        print("Fehler beim Laden des Kaskadenklassifikators für die Gesichtserkennung.")
else:
        print("Kaskadenklassifikator erfolgreich geladen.")

#Detect all faces in image.
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

#Draw a rectangle over the face, and detect eyes in faces
for (x,y,w,h) in faces:
    cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

    #ROI is region of interest with area having face inside it.
    roi_gray = gray[y:y+h, x:x+w]
    roi_gray = cv.blur(roi_gray, (10, 10))
    roi_color = img[y:y+h, x:x+w]

    #Detect eyes in face
    eyes = eye_cascade.detectMultiScale(roi_gray)

    for (ex,ey,ew,eh) in eyes:
        #cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        cv.circle(roi_color,(ex+ int(ew/2) , ey + int(eh/2) ),10,(0,0,255),1)




roi_HSV = cv.cvtColor(roi_color, cv.COLOR_BGR2HSV)
cv.imshow('HSV', roi_HSV)

# define HSV color ranges for eyes colors
class_name = ("Blue", "Brown", "Green")
EyeColor = {
    class_name[0] : ((180, 20, 50), (255, 100, 100)), 
    class_name[1] : ((2, 20, 20), (40, 60, 100)),    
    class_name[2] : ((70, 40, 50), (165, 100, 100)),
}
def match_color(hsv_value):
    for color_class, (lower, upper) in EyeColor.items():
        if lower[0] <= hsv_value[0] <= upper[0] and lower[1] <= hsv_value[1] <= upper[1] and lower[2] <= hsv_value[2] <= upper[2]:
            return color_class
    return "Other"

for(ex,ey,ew,eh) in eyes:
    cv.circle(roi_color,(ex+ int(ew/2) , ey + int(eh/2) ),10,(0,0,255),1)
    center = (ex+ int(ew/2) , ey + int(eh/2) )
    radius = 10
    mask_circle = np.zeros_like(roi_HSV[:, :, 0])
    cv.circle(mask_circle, center, radius, 255, -1)
    masked_image = cv.bitwise_and(roi_HSV, roi_HSV, mask=mask_circle)
    cv.imshow("Maskiertes Bild", masked_image)
    mean_hsv = cv.mean(masked_image)
    print(mean_hsv)
    eye_color = match_color(mean_hsv[:3])
    print("Erkannte Augenfarbe:", eye_color)

    
cv.imshow('Image', img)
cv.waitKey(0)
cv.destroyAllWindows()

Kaskadenklassifikator erfolgreich geladen.
(0.23987727313446205, 0.2902893487413778, 0.3229120009555376, 0.0)
Erkannte Augenfarbe: Other
(0.24348293469497448, 0.19913255098689123, 0.23477858401266088, 0.0)
Erkannte Augenfarbe: Other
